In [156]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import numpy as np
import pandas as pd
import textwrap
from pprint import pprint
import os
import re
import data_processing
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\arthu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [157]:
path_to_file = "acerpi_dataset/test/"

#### Open Unannotated Documents
We get the unnanotatted texts and store each document as a key, value pair in a dictionary

In [158]:
documents = {}
for filename in os.listdir(path_to_file):
   if filename.endswith(".txt"):
      with open(os.path.join(path_to_file, filename), 'r', encoding="utf8") as f:
         key = int(filename.replace('.txt', ''))
         value = f.read()
         documents[key] = value

Next we store the documents in a DataFrame

In [159]:
df_docs = pd.DataFrame.from_dict(documents, orient='index')
df_docs.columns = ['text']
df_docs.head()

,text
105798,"Documento gerado sob autenticação Nº LKB.506.405.IRF, disponível no endereço\nhttp://www.ufrgs.br/autenticacao\nDocumento certificado eletronicamente, conforme Portaria nº\n3362/2016, que institui o Sistema de Documentos Eletrônicos da UFRGS.\n1/1\nPORTARIA Nº 1955 de 05/03/2020\nO PRÓ-REITOR DE GESTÃO DE PESSOAS DA UNIVERSIDADE FEDERAL DO RIO GRANDE DO SUL, no\nuso de suas atribuições que lhe foram conferidas pela Portaria nº.7684, de 03 de outubro de 2016, do\nMagnífico Reitor, e conforme o Laudo Médico n°60131,\nRESOLVE\nDesignar, temporariamente, nos termos da Lei nº. 8.112, de 11 de dezembro de 1990, com redação\ndada pela Lei nº.9.527, de 10 de dezembro de 1997, a ocupante do cargo de ASSISTENTE EM\nADMINISTRAÇÃO, do Quadro de Pessoal desta Universidade, FRANCIELE MARQUES ZIQUINATTI (Siape:\n1091092 ), para substituir TURENE ANDRADE E SILVA NETO (Siape: 0356721 ), Diretor da Divisão de Preparo\nda Licitação do Departamento de Aquisição de Bens e Serviços, Código FG-4, em seu afastamento por motivo\nde Laudo Médico do titular da Função, no período de 30/01/2020 a 15/03/2020, com o decorrente\npagamento das vantagens por 46 dias.\nMAURÍCIO VIÉGAS DA SILVA\nPró-Reitor de Gestão de Pessoas\n"
105799,"Documento gerado sob autenticação Nº BOA.507.615.IRF, disponível no endereço\nhttp://www.ufrgs.br/autenticacao\nDocumento certificado eletronicamente, conforme Portaria nº\n3362/2016, que institui o Sistema de Documentos Eletrônicos da UFRGS.\n1/1\nPORTARIA Nº 1956 de 05/03/2020\nO PRÓ-REITOR DE GESTÃO DE PESSOAS DA UNIVERSIDADE FEDERAL DO RIO GRANDE DO SUL, no\nuso de suas atribuições que lhe foram conferidas pela Portaria nº.7684, de 03 de outubro de 2016, do\nMagnífico Reitor, e conforme o Laudo Médico n°60170,\nRESOLVE\nDesignar, temporariamente, nos termos da Lei nº. 8.112, de 11 de dezembro de 1990, com redação\ndada pela Lei nº.9.527, de 10 de dezembro de 1997, o ocupante do cargo de TÉCNICO EM ASSUNTOS\nEDUCACIONAIS, do Quadro de Pessoal desta Universidade, FELIPE ANTÔNIO GUIDI SAUERESSIG (Siape:\n1963089 ), para substituir JOSE EDUARDO HANDEL (Siape: 0357511 ), Secretário da COMGRAD de Nutrição,\nCódigo FG-7, em seu afastamento por motivo de Laudo Médico do titular da Função, no período de\n05/02/2020 a 11/02/2020, com o decorrente pagamento das vantagens por 7 dias.\nMAURÍCIO VIÉGAS DA SILVA\nPró-Reitor de Gestão de Pessoas\n"
105801,"Documento gerado sob autenticação Nº KMT.508.839.IRF, disponível no endereço\nhttp://www.ufrgs.br/autenticacao\nDocumento certificado eletronicamente, conforme Portaria nº\n3362/2016, que institui o Sistema de Documentos Eletrônicos da UFRGS.\n1/1\nPORTARIA Nº 1957 de 05/03/2020\nO PRÓ-REITOR DE GESTÃO DE PESSOAS DA UNIVERSIDADE FEDERAL DO RIO GRANDE DO SUL, no\nuso de suas atribuições que lhe foram conferidas pela Portaria nº.7684, de 03 de outubro de 2016, do\nMagnífico Reitor, e conforme o Laudo Médico n°60148,\nRESOLVE\nDesignar, temporariamente, nos termos da Lei nº. 8.112, de 11 de dezembro de 1990, com redação\ndada pela Lei nº.9.527, de 10 de dezembro de 1997, o ocupante do cargo de TÉCNICO DE LABORATÓRIO\nÁREA, do Quadro de Pessoal desta Universidade, PEDRO MOTIFUMI KUAMOTO (Siape: 0354874 ), para\nsubstituir Ricardo Schäffer da Rosa (Siape: 1935227 ), Coordenador do Núcleo de Infraestrutura da Gerência\nAdm do Instituto de Química, Código FG-7, em seu afastamento por motivo de Laudo Médico do titular da\nFunção, no dia 13/02/2020, com o decorrente pagamento das vantagens por 1 dia.\nMAURÍCIO VIÉGAS DA SILVA\nPró-Reitor de Gestão de Pessoas\n"
105802,"Documento gerado sob autenticação Nº DYG.509.949.IRF, disponível no endereço\nhttp://www.ufrgs.br/autenticacao\nDocumento certificado eletronicamente, conforme Portaria nº\n3362/2016, que institui o Sistema de Documentos Eletrônicos da UFRGS.\n1/1\nPORTARIA Nº 1958 de 05/03/2020\nO PRÓ-REITOR DE GESTÃO DE PESSOAS DA UNIVERSIDADE FEDERAL DO RIO GRANDE DO SUL, no\nuso de suas atribuições que lhe foram conferidas pela Por

#### Data Cleaning
We will use the **data_processing.clear_text** function which was built to clean the specific document we are using here. Since we already know they will be used for NER, the objective of the cleaning is getting the text into a format which will be easy to split into sentences. Each sentence will later be used as an input to our neural network.
There are three aspects we are cleaning:
1. Removing line breaks as they make it very hard for the sentence tokenizer to correctly recognize the begining and ending of sentences in the text
2. Removing repeated whitespaces which are used for the visual formatting of the documents but will generate unnecessary tokens for our neural network.
3. There are a lot of law and document numbers in this documents. There isn't a consistent writing of these numbers which can start as "nº.123", "nº. 123" and "nº 123". We will padronize this occurences to appear as "nº 123" since the punctuation after "nº" makes it harder for the sentence tokenizer to correctly separate the sentences.

In [160]:
print(df_docs['text'].iloc[0])
df_docs['text'] = df_docs['text'].apply(data_processing.clear_text)
print(df_docs['text'].iloc[0])

Documento gerado sob autenticação Nº LKB.506.405.IRF, disponível no endereço
http://www.ufrgs.br/autenticacao
Documento certificado eletronicamente, conforme Portaria nº
3362/2016, que institui o Sistema de Documentos Eletrônicos da UFRGS.
1/1
PORTARIA Nº             1955                  de  05/03/2020
O PRÓ-REITOR DE GESTÃO DE PESSOAS DA UNIVERSIDADE FEDERAL DO RIO GRANDE DO SUL, no
uso de suas atribuições que lhe foram conferidas pela Portaria nº.7684, de 03 de outubro de 2016, do
Magnífico Reitor, e conforme o Laudo Médico n°60131,
RESOLVE
Designar, temporariamente, nos termos da Lei nº. 8.112, de 11 de dezembro de 1990, com redação
dada  pela  Lei  nº.9.527,  de  10  de  dezembro  de  1997,  a  ocupante  do  cargo  de  ASSISTENTE  EM
ADMINISTRAÇÃO, do Quadro de Pessoal  desta Universidade,  FRANCIELE MARQUES ZIQUINATTI (Siape:
1091092 ),  para substituir   TURENE ANDRADE E SILVA NETO (Siape: 0356721 ), Diretor da Divisão de Preparo
da Licitação do Departamento de Aquisição de Bens

#### Sentence tokenization
Having cleaned our documents we will split them into sentences. The **data_processing.split_text_sentences** function will return a DataFrame with: The sentences, a unique ID for each sentence and the index of the document each sentence is a part of.

In [161]:
df_sentences = data_processing.split_text_sentences(df_docs['text'])
pd.set_option("display.max_colwidth", 0)
df_sentences.head()

,document,sentence,sentence_id
0,105798,"Documento gerado sob autenticação Nº LKB.506.405.IRF, disponível no endereço http://www.ufrgs.br/autenticacao Documento certificado eletronicamente, conforme Portaria nº 3362/2016, que institui o Sistema de Documentos Eletrônicos da UFRGS.",0
1,105798,"1/1 PORTARIA Nº 1955 de 05/03/2020 O PRÓ-REITOR DE GESTÃO DE PESSOAS DA UNIVERSIDADE FEDERAL DO RIO GRANDE DO SUL, no uso de suas atribuições que lhe foram conferidas pela Portaria nº 7684, de 03 de outubro de 2016, do Magnífico Reitor, e conforme o Laudo Médico n°60131, RESOLVE Designar, temporariamente, nos termos da Lei nº 8.112, de 11 de dezembro de 1990, com redação dada pela Lei nº 9.527, de 10 de dezembro de 1997, a ocupante do cargo de ASSISTENTE EM ADMINISTRAÇÃO, do Quadro de Pessoal desta Universidade, FRANCIELE MARQUES ZIQUINATTI (Siape: 1091092 ), para substituir TURENE ANDRADE E SILVA NETO (Siape: 0356721 ), Diretor da Divisão de Preparo da Licitação do Departamento de Aquisição de Bens e Serviços, Código FG-4, em seu afastamento por motivo de Laudo Médico do titular da Função, no período de 30/01/2020 a 15/03/2020, com o decorrente pagamento das vantagens por 46 dias.",1
2,105798,MAURÍCIO VIÉGAS DA SILVA Pró-Reitor de Gestão de Pessoas,2
3,105799,"Documento gerado sob autenticação Nº BOA.507.615.IRF, disponível no endereço http://www.ufrgs.br/autenticacao Documento certificado eletronicamente, conforme Portaria nº 3362/2016, que institui o Sistema de Documentos Eletrônicos da UFRGS.",3
4,105799,"1/1 PORTARIA Nº 1956 de 05/03/2020 O PRÓ-REITOR DE GESTÃO DE PESSOAS DA UNIVERSIDADE FEDERAL DO RIO GRANDE DO SUL, no uso de suas atribuições que lhe foram conferidas pela Portaria nº 7684, de 03 de outubro de 2016, do Magnífico Reitor, e conforme o Laudo Médico n°60170, RESOLVE Designar, temporariamente, nos termos da Lei nº 8.112, de 11 de dezembro de 1990, com redação dada pela Lei nº 9.527, de 10 de dezembro de 1997, o ocupante do cargo de TÉCNICO EM ASSUNTOS EDUCACIONAIS, do Quadro de Pessoal desta Universidade, FELIPE ANTÔNIO GUIDI SAUERESSIG (Siape: 1963089 ), para substituir JOSE EDUARDO HANDEL (Siape: 0357511 ), Secretário da COMGRAD de Nutrição, Código FG-7, em seu afastamento por motivo de Laudo Médico do titular da Função, no período de 05/02/2020 a 11/02/2020, com o decorrente pagamento das vantagens por 7 dias.",4


#### Dealing with duplicate sentences
Now that we have our dataset of sentences we'll go to the last step which is identifying and categorizing duplicate sentences. Since we need to manually annotate the dataset for training we can group duplicate sentences to facilitate the anottation process. The document we have are fairly standardized so there are a few duplicates we will find.

Let's group sentences by their textual content. Each group will be composed of identical sentences.

In [162]:
test_group = df_sentences.groupby('sentence')

In [163]:
#pd.set_option("display.max_rows", 400)
#test_group.describe()

We loop each group of duplicated sentences and create a dictionary where the key will be the index of the first sentence of the group and the value is the index of all sentences equal to the first.

In [164]:
duplicates = {}
for i in test_group:
    dup_list = i[1]['sentence_id'].tolist()
    duplicates[dup_list[0]] = dup_list


We create a dataframe which will contain only unique sentences. The sentences that have identic pairs will have all of their indexes in a list in the 'duplicates' column. This will allow us to replicate the labels after the anotattion process.

In [181]:
df_unique_sentences = df_sentences.copy()
df_unique_sentences['duplicates'] = pd.Series(duplicates, index=df_sentences.index)
df_unique_sentences = df_unique_sentences.dropna(subset=['duplicates'])
df_unique_sentences['label'] = ""

We can see that the unique dataframe has less sentences which represen the duplicates we removed

In [182]:
pprint(df_sentences.shape)
pprint(df_unique_sentences.shape)

(106, 3)
(84, 5)


In [183]:
df_unique_sentences.head(10)

,document,sentence,sentence_id,duplicates,label
0,105798,"Documento gerado sob autenticação Nº LKB.506.405.IRF, disponível no endereço http://www.ufrgs.br/autenticacao Documento certificado eletronicamente, conforme Portaria nº 3362/2016, que institui o Sistema de Documentos Eletrônicos da UFRGS.",0,[0],
1,105798,"1/1 PORTARIA Nº 1955 de 05/03/2020 O PRÓ-REITOR DE GESTÃO DE PESSOAS DA UNIVERSIDADE FEDERAL DO RIO GRANDE DO SUL, no uso de suas atribuições que lhe foram conferidas pela Portaria nº 7684, de 03 de outubro de 2016, do Magnífico Reitor, e conforme o Laudo Médico n°60131, RESOLVE Designar, temporariamente, nos termos da Lei nº 8.112, de 11 de dezembro de 1990, com redação dada pela Lei nº 9.527, de 10 de dezembro de 1997, a ocupante do cargo de ASSISTENTE EM ADMINISTRAÇÃO, do Quadro de Pessoal desta Universidade, FRANCIELE MARQUES ZIQUINATTI (Siape: 1091092 ), para substituir TURENE ANDRADE E SILVA NETO (Siape: 0356721 ), Diretor da Divisão de Preparo da Licitação do Departamento de Aquisição de Bens e Serviços, Código FG-4, em seu afastamento por motivo de Laudo Médico do titular da Função, no período de 30/01/2020 a 15/03/2020, com o decorrente pagamento das vantagens por 46 dias.",1,[1],
2,105798,MAURÍCIO VIÉGAS DA SILVA Pró-Reitor de Gestão de Pessoas,2,"[2, 5, 8, 11, 17, 19, 21, 23, 25, 41, 56]",
3,105799,"Documento gerado sob autenticação Nº BOA.507.615.IRF, disponível no endereço http://www.ufrgs.br/autenticacao Documento certificado eletronicamente, conforme Portaria nº 3362/2016, que institui o Sistema de Documentos Eletrônicos da UFRGS.",3,[3],
4,105799,"1/1 PORTARIA Nº 1956 de 05/03/2020 O PRÓ-REITOR DE GESTÃO DE PESSOAS DA UNIVERSIDADE FEDERAL DO RIO GRANDE DO SUL, no uso de suas atribuições que lhe foram conferidas pela Portaria nº 7684, de 03 de outubro de 2016, do Magnífico Reitor, e conforme o Laudo Médico n°60170, RESOLVE Designar, temporariamente, nos termos da Lei nº 8.112, de 11 de dezembro de 1990, com redação dada pela Lei nº 9.527, de 10 de dezembro de 1997, o ocupante do cargo de TÉCNICO EM ASSUNTOS EDUCACIONAIS, do Quadro de Pessoal desta Universidade, FELIPE ANTÔNIO GUIDI SAUERESSIG (Siape: 1963089 ), para substituir JOSE EDUARDO HANDEL (Siape: 0357511 ), Secretário da COMGRAD de Nutrição, Código FG-7, em seu afastamento por motivo de Laudo Médico do titular da Função, no período de 05/02/2020 a 11/02/2020, com o decorrente pagamento das vantagens por 7 dias.",4,[4],
6,105801,"Documento gerado sob autenticação Nº KMT.508.839.IRF, disponível no endereço http://www.ufrgs.br/autenticacao Documento certificado eletronicamente, conforme Portaria nº 3362/2016, que institui o Sistema de Documentos Eletrônicos da UFRGS.",6,[6],
7,105801,"1/1 PORTARIA Nº 1957 de 05/03/2020 O PRÓ-REITOR DE GESTÃO DE PESSOAS DA UNIVERSIDADE FEDERAL DO RIO GRANDE DO SUL, no uso de suas atribuições que lhe foram conferidas pela Portaria nº 7684, de 03 de outubro de 2016, do Magnífico Reitor, e conforme o Laudo Médico n°60148, RESOLVE Designar, temporariamente, nos termos da Lei nº 8.112, de 11 de dezembro de 1990, com redação dada pela Lei nº 9.527, de 10 de dezembro de 1997, o ocupante do cargo de TÉCNICO DE LABORATÓRIO ÁREA, do Quadro de Pessoal desta Universidade, PEDRO MOTIFUMI KUAMOTO (Siape: 0354874 ), para substituir Ricardo Schäffer da Rosa (Siape: 1935227 ), Coordenador do Núcleo de Infraestrutura da Gerência Adm do Instituto de Química, Código FG-7, em seu afastamento por motivo de Laudo Médico do titular da Função, no dia 13/02/2020, com o decorrente pagamento das vantagens por 1 dia.",7,[7],
9,105802,"Documento gerado sob autenticação Nº DYG.509.949.IRF, disponível no endereço http://www.ufrgs.br/autenticacao Documento certificado eletronicamente, conforme Portaria nº 3362/2016, que institui o Sistema de Documentos Eletrônicos da UFRGS.",9,[9],
10,105802,"1/1 PORTARIA Nº 1958 de 05/03/2020 O PRÓ-REITOR DE GESTÃO DE PESSOAS DA UNIVERSIDADE FEDERAL DO RIO GRANDE DO SUL, no uso de suas atribuições que lhe foram c

In [184]:
#pprint(df_unique_sentences.to_json(lines=True, orient = 'records'))
df_unique_sentences.to_json(os.path.join(path_to_file, 'unique_sentences.jsonl'),lines=True, orient = 'records')